In [1]:
# Start final project import modules
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
%matplotlib inline
import sys
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder 
from sklearn import feature_extraction
from __future__ import print_function
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


In [2]:
# read in the datasets
 
#jd = pd.read_csv('job_data_dict_30_sample.csv')
#jd = pd.read_csv('job_data_dict_sal_weighted_30_sample.csv')
#jd = pd.read_csv('job_data_dict_all_sal_weighted.csv')
jd = pd.read_csv('job_data_dict_all_sal_weighted_major.csv')

In [3]:
#1. Explore the dataset: 
    # There is a mix of string and numeric data. There are some variables that can be converted to dummy or binary values.
    # While there are no NAs there are unknown values in some of the string data which may need to be treated as NAs.

#jd.describe(include='all')
jd.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9716 entries, 0 to 9715
Data columns (total 12 columns):
Unnamed: 0          9716 non-null int64
major_des           9716 non-null object
jobs                9716 non-null object
jobtitle            9716 non-null object
onetcode            9716 non-null object
major_code          9716 non-null int64
dictionary          9716 non-null object
p_an                9716 non-null float64
p_an_normal         9716 non-null float64
lowct               9716 non-null int64
major_dictionary    9716 non-null object
uid                 9716 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 986.8+ KB


In [4]:
jd.head()

,Unnamed: 0,major_des,jobs,jobtitle,onetcode,major_code,dictionary,p_an,p_an_normal,lowct,major_dictionary,uid
0,2,Management,computer and information systems managers,director of it planning development deployment,11-3021.00,11,director planning development deployment direc...,110000.0,0.097679,0,computer information systems managers computer...,1
1,6,Management,financial managers branch or department,accounting manager supervisor,11-3031.02,11,accounting manager supervisor accounting manag...,45000.0,0.031779,0,financial managers branch department financial...,2
2,16,Management,human resources managers,human resources manager,11-3040.00,11,human resources manager human resources manage...,91197.5,0.078616,0,human resources managers human resources manag...,3
3,28,Management,treasurers and controllers,corporate controller,11-3031.01,11,corporate controller corporate controller year...,115000.0,0.102748,0,treasurers controllers treasurers controllers ...,4
4,31,Management,financial managers branch or department,accounting manager supervisor,11-3031.02,11,accounting manager supervisor accounting manag...,82500.0,0.069798,0,financial managers branch department financial...,5


In [5]:

cat_columns = ['major_des', 'jobs','jobtitle','dictionary','major_dictionary', 'p_an_normal']
major_feat_col = ['major_dictionary', 'p_an_normal']   
feat_col = ['dictionary', 'p_an_normal']
#feat_col = ['jobtitle', 'p_an_normal']
string_variables = ['jobs']

jd_2 = jd[cat_columns]
jd_2.head()

,major_des,jobs,jobtitle,dictionary,major_dictionary,p_an_normal
0,Management,computer and information systems managers,director of it planning development deployment,director planning development deployment direc...,computer information systems managers computer...,0.097679
1,Management,financial managers branch or department,accounting manager supervisor,accounting manager supervisor accounting manag...,financial managers branch department financial...,0.031779
2,Management,human resources managers,human resources manager,human resources manager human resources manage...,human resources managers human resources manag...,0.078616
3,Management,treasurers and controllers,corporate controller,corporate controller corporate controller year...,treasurers controllers treasurers controllers ...,0.102748
4,Management,financial managers branch or department,accounting manager supervisor,accounting manager supervisor accounting manag...,financial managers branch department financial...,0.069798


In [6]:
# subset the data to the major groups
#jd_3 = jd_2[jd_2.major_des=='Management']

In [7]:
jd_2.major_des.value_counts().min()

2

In [1]:
#df_numerical = jd_2.copy()
#encoder_dict = {}

#for col in string_variables:
#    le = preprocessing.LabelEncoder()
#    le.fit(jd_2[col])
#    encoder_dict[col] = le
#    df_numerical[col] = le.transform(jd_2[col])

#df_numerical.head()

In [9]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(jd_2[major_feat_col], jd_2['major_des'], random_state=1)

In [10]:
X_test.shape

(2429L, 2L)

In [11]:
X_train.shape

(7287L, 2L)

In [12]:
type(X_train)

numpy.ndarray

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(decode_error = 'ignore')
#vect = CountVectorizer()
#vect.fit(X_train)
#vect.transform(X_train)

In [14]:
#vect.get_feature_names()
# learn vocabulary and create document-term matrix in a single step
train_dtm = vect.fit_transform(X_train[:,0])
train_dtm

<7287x6219 sparse matrix of type '<type 'numpy.int64'>'
	with 90302 stored elements in Compressed Sparse Row format>

In [15]:
# transform testing data into a document-term matrix
test_dtm = vect.transform(X_test[:,0])
type(test_dtm)

scipy.sparse.csr.csr_matrix

In [16]:
train_features = vect.get_feature_names()
len(train_features)

6219

In [17]:
train_features = train_features + list('s')

In [18]:
len(train_features)

6220

In [19]:
train_sal = np.array(X_train[:,1])
train_sal = train_sal[...,None]
type(train_sal)

numpy.ndarray

In [20]:
test_sal = np.array(X_test[:,1])
test_sal = test_sal[...,None]
type(test_sal)

numpy.ndarray

In [21]:
train_dtm_2 = np.array(train_dtm.todense())
type(train_dtm_2)

numpy.ndarray

In [22]:
test_dtm_2 = np.array(test_dtm.todense())

In [23]:
test_dtm_2.shape

(2429L, 6219L)

In [24]:
train_dtm_2.shape

(7287L, 6219L)

In [25]:
train_dtm2 = np.append(train_dtm_2, train_sal,1)

In [26]:
type(train_dtm2)
train_dtm2.shape

(7287L, 6220L)

In [27]:
test_dtm2 = np.append(test_dtm_2, test_sal, 1)

In [28]:
# convert train_dtm to a regular array
#train_arr = train_dtm2.toarray()
#train_arr

In [29]:
# exercise: create a DataFrame of tokens with their counts.
train_token_counts = pd.DataFrame({'token':train_features, 'count':np.sum(train_dtm2, axis=0)})
train_token_counts.sort_index(by='count', ascending=False)

,count,token
1169,2412,computer
3336,2302,managers
3333,2071,manager
4927,1833,sales
1906,1778,engineers
5227,1546,software
5078,1529,senior
5542,1442,systems
1002,1358,clerks
1902,1282,engineer


In [30]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm2, y_train)

C:\Anaconda\lib\site-packages\sklearn\naive_bayes.py:427: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  self.feature_count_ += safe_sparse_dot(Y.T, X)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [31]:
# make predictions on test data using test_dtm
preds = nb.predict(test_dtm2)
preds

array(['Management', 'Management',
       'Healthcare Practitioners and Technical', ...,
       'Architecture and Engineering', 'Architecture and Engineering',
       'Computer and Mathematical'], 
      dtype='|S46')

In [32]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.957595718403
[[144   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0  58   0   0   0   1   0   0   0   0   0   1   0   0   2   1   0   0
    0   0   0]
 [  0   0   1   0   0   0   0   0   0   0   0   1   0   0   0   0   0   1
    0   0   0]
 [  0   0   0 236   0   5   0   0   0   0   0   0   0   0   2   2   0   0
    0   0   0]
 [  0   0   0   0  14   0   0   0   0   0   0   0   0   0   0   0   0   1
    0   0   0]
 [  0   0   0   2   0 540   0   0   0   0   0   0   0   0  12   2   0   0
    0   0   0]
 [  2   0   0   0   0   0   7   0   0   0   0   0   0   0   0   0   0   1
    0   0   0]
 [  0   0   0   0   0   0   0   3   0   1   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   4   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  1   1   0   2   0   0   0   0   0  55   0   0   0   0   3   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   8   0   0   0   0   4   0   0
    0   0   0]
 [  0 

In [38]:
# exercise: show the message text for the false positives
#X_test[y_test < preds]

In [39]:

# exercise: show the message text for the false negatives
#X_test[y_test > preds]

In [33]:
from sklearn.metrics import classification_report 

print(classification_report(y_test, preds) )

             precision    recall  f1-score   support

Architecture and Engineering       0.95      0.99      0.97       145
Arts, Design, Entertainment, Sports, and Media       0.95      0.92      0.94        63
Building and Grounds Cleaning and Maintenance       1.00      0.33      0.50         3
Business and Financial Operations       0.97      0.96      0.97       245
Community and Social Service       0.88      0.93      0.90        15
Computer and Mathematical       0.96      0.97      0.96       556
Construction and Extraction       1.00      0.70      0.82        10
Education, Training, and Library       0.75      0.75      0.75         4
Food Preparation and Serving Related       1.00      1.00      1.00         4
Healthcare Practitioners and Technical       0.93      0.89      0.91        62
Healthcare Support       0.89      0.67      0.76        12
Installation, Maintenance, and Repair       0.93      0.98      0.96        44
      Legal       1.00      0.88      0.93       

C:\Anaconda\lib\site-packages\sklearn\metrics\metrics.py:1771: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
# use SVM
from sklearn import svm
svc = svm.SVC(kernel='linear', probability = True)

svc.fit(train_dtm2, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [35]:
# make predictions on test data using test_dtm
preds = svc.predict(test_dtm2)
preds

array(['Management', 'Management',
       'Healthcare Practitioners and Technical', ...,
       'Architecture and Engineering', 'Architecture and Engineering',
       'Computer and Mathematical'], dtype=object)

In [36]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.993001235076
[[145   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0  63   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1
    0   0   0]
 [  0   0   0 244   0   0   0   0   0   0   0   0   0   0   0   1   0   0
    0   0   0]
 [  0   0   0   0  14   0   0   0   0   0   0   0   0   1   0   0   0   0
    0   0   0]
 [  0   0   0   0   0 556   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   1   0   0   9   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   1   0   0   0   3   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   1   0   0   0   0   3   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   2   0   2   0   0   0   0   0  58   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  12   0   0   0   0   0   0   0
    0   0   0]
 [  0 

In [147]:
# format the data so I can run the cross validation

In [45]:
train_dtm = vect.fit_transform(jd_2['major_dictionary'])
train_dtm_2 = np.array(train_dtm.todense())
type(train_dtm_2)

numpy.ndarray

In [46]:
train_sal = np.array(jd_2['p_an_normal'])
train_sal = train_sal[...,None]
type(train_sal)

numpy.ndarray

In [47]:
train_dtm2 = np.append(train_dtm_2, train_sal,1)

In [48]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm2, jd_2['major_des'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [49]:
preds = nb.predict(train_dtm2)
preds.shape

(9716L,)

In [50]:

print(classification_report(jd_2['major_des'], preds) )

             precision    recall  f1-score   support

Architecture and Engineering       0.98      0.99      0.99       572
Arts, Design, Entertainment, Sports, and Media       0.99      0.97      0.98       270
Building and Grounds Cleaning and Maintenance       1.00      0.94      0.97        17
Business and Financial Operations       0.98      0.99      0.99      1011
Community and Social Service       0.97      0.98      0.98        63
Computer and Mathematical       0.98      0.97      0.97      2251
Construction and Extraction       1.00      0.94      0.97        36
Education, Training, and Library       1.00      0.81      0.90        16
Farming, Fishing, and Forestry       0.00      0.00      0.00         2
Food Preparation and Serving Related       1.00      0.97      0.99        34
Healthcare Practitioners and Technical       0.96      0.99      0.97       201
Healthcare Support       0.95      0.74      0.83        54
Installation, Maintenance, and Repair       0.97      0.

In [51]:

target_predicted = nb.predict(train_dtm2)
target_prob = nb.predict_proba(train_dtm2)

#print(target_test)
#print(target_predicted)
#print(target_prob.mean())

pd.crosstab(jd_2['major_des'], target_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,Architecture and Engineering,"Arts, Design, Entertainment, Sports, and Media",Building and Grounds Cleaning and Maintenance,Business and Financial Operations,Community and Social Service,Computer and Mathematical,Construction and Extraction,"Education, Training, and Library",Food Preparation and Serving Related,Healthcare Practitioners and Technical,...,Legal,"Life, Physical, and Social Science",Management,Office and Administrative Support,Personal Care and Service,Production,Protective Service,Sales and Related,Transportation and Material Moving,All
True,,,,,,,,,,,,,,,,,,,,,
Architecture and Engineering,569,0,0,0,0,2,0,0,0,0,...,0,0,1,0,0,0,0,0,0,572
"Arts, Design, Entertainment, Sports, and Media",0,263,0,0,0,1,0,0,0,0,...,0,0,4,0,0,0,0,1,0,270
Building and Grounds Cleaning and Maintenance,0,0,16,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,17
Business and Financial Operations,1,0,0,1001,0,1,0,0,0,0,...,0,0,3,3,0,1,0,0,0,1011
Community and Social Service,0,0,0,0,62,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,63
Computer and Mathematical,0,2,0,16,1,2175,0,0,0,1,...,0,1,53,1,0,1,0,0,0,2251
Construction and Extraction,0,0,0,0,0,0,34,0,0,0,...,0,0,0,0,0,2,0,0,0,36
"Education, Training, and Library",1,0,0,0,1,1,0,13,0,0,...,0,0,0,0,0,0,0,0,0,16
"Farming, Fishing, and Forestry",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,2


In [52]:
from sklearn.cross_validation import cross_val_score
cross_val_score(nb, train_dtm, jd_2['major_des'], cv=5, scoring='accuracy').mean()

C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:413: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


0.95974677616614001